<a href="https://colab.research.google.com/github/ShahZebYousafzai/Deep-Learning-Basics/blob/main/1_tensorflow_fundementals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covering fundemental concepts using Tensorflow

We are going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors and Numpy
* Using @tf.function (A way to speed up your regular Python functions)
* Using GPUs with Tensorflow (or TPUs)

## Introduction to Tensors

In [ ]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

2.6.0


In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check number of dimension of a tensor (ndim)
scalar.ndim

0

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
# Check the dimension of vector
vector.ndim

1

In [ ]:
# Create a matrix
matrix = tf.constant([[10, 7],
                      [10, 7]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [10,  7]], dtype=int32)>

In [ ]:
# Check matrix dimension
matrix.ndim

2

In [ ]:
# Create another matrix
another_matrix = tf.constant([[10., 1.],
                               [23., 12.],
                               [9., 1.]], dtype=tf.float16) # Specify data type to tensor
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  1.],
       [23., 12.],
       [ 9.,  1.]], dtype=float16)>

In [ ]:
another_matrix.ndim

2

In [ ]:
# Let us create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                       [[7, 8, 9],
                       [10, 11, 12]],
                       [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [ ]:
tensor.ndim

3

What we have created so far:

* Scalar: A single number
* Vector: A number with direction (e.g. wind speed and direction)
* Matrix: A 2-dimensional array of numbers
* Tensor: An n-dimension array of numbers (When n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable`

In [ ]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Let us try changing one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: ignored

In [ ]:
# How about we try .assign()
changeable_tensor[0].assign(7)

In [ ]:
# Let us try our unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

**Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as Tensorflow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

Random tensors are tensors of random arbitrary size which contain random numbers

In [ ]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproductibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

# Are they equal
random_1, random_2, random_1 == random_2

In [ ]:
# Create two random (but the same) tensors with different seed
random_1 = tf.random.Generator.from_seed(7) # set seed for reproductibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3,2))

# Are they equal
random_1, random_2, random_1 == random_2

### Shuffle the order of elements in tensors

In [ ]:
# Shuffle a tensor (valuable for when you want to shuffle your data so inherent order doesent effect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
not_shuffled.ndim

In [ ]:
# shuffle out non-shuffled tensor
shuffled = tf.random.shuffle(not_shuffled)
shuffled

In [ ]:
# shuffle out non-shuffled tensor with seed
shuffled = tf.random.shuffle(not_shuffled, seed=42)
shuffled

In [ ]:
# shuffle out non-shuffled tensor with seed
tf.random.set_seed(42)
shuffled = tf.random.shuffle(not_shuffled, seed=42)
shuffled

In [ ]:
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level seed

It looks like if we want our shuffle tensors to be in the same order, we've got to use the global level random seed as well the operation level random seed:

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

### Other ways to create tensors

In [ ]:
tf.ones([10, 7])

In [ ]:
# create a tensor of all zeros
tf.zeros(shape=(3,4))

### We can also turn numpy arrays into tensors

The main difference between numpy arrays and Tensorflow tensors is that tensors can be run on a GPU (much faster for numerical computing).

In [ ]:
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # Create numpy array of 1 and 25
numpy_A

In [ ]:
# convert numpy array to tensors
A = tf.constant(numpy_A)
A

In [ ]:
# changing the shape of tensor
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A)
A, B

In [ ]:
# changing the shape of tensor
A = tf.constant(numpy_A, shape=(2, 3, 5))
B = tf.constant(numpy_A)
A, B

⚠️ The above doesn't work as the element in `numpy_A` is 24 and we are reshaping to (2, 3, 5) = 30

In [ ]:
A.ndim

### Getting information from tensors

When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [ ]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

In [ ]:
rank_4_tensor[0]

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

In [ ]:
2 * 3 * 4 * 5

In [ ]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in tensor:", tf.size(rank_4_tensor))
print("Total number of elements in tensor:", tf.size(rank_4_tensor).numpy())

### Indexing Tensors

Tensors can be indexed just like Python lists

In [ ]:
some_list = [1, 2, 3, 4]
some_list[:2]

In [ ]:
# Get the first 2 elements of each dimension of our tensor
rank_4_tensor[:2, :2, :2, :2]

In [ ]:
# Get first elemet from each dimension from each index except the final one
rank_4_tensor[:1, :1, :1] # or rank_4_tensor[:1, :1, :1, :]

In [ ]:
rank_4_tensor[:1, :1, :1, :]

In [ ]:
rank_4_tensor[:1, :1, :, :1]

In [ ]:
rank_4_tensor[:1, :, :1, :1]

In [ ]:
rank_4_tensor[:, :1, :1, :1]

In [ ]:
# Create a rank 2 tensor (2 dimension)
rank_2_tensor = tf.constant([[10, 4],
                             [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

In [ ]:
rank_2_tensor

In [ ]:
some_list, some_list[-1]

In [ ]:
# Get the last item of each of row of our rank 2 tensor
rank_2_tensor[:, -1]

In [ ]:
# Add extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # or rank_3_tensor = rank_2_tensor[:, :, tf.newaxis]
rank_3_tensor

In [ ]:
# So the other alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # axis=-1 means the last dimension

In [ ]:
# So the other alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=0) # axis=0 means the first dimension

In [ ]:
# So the other alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=1) # axis=1 means the second dimension

In [ ]:
rank_2_tensor

### Manipulating tensors (tensor operations)

** Basic Operations**

`+`, `-`, `*`, `/`

In [ ]:
# You can add valuse to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

In [ ]:
# Original tensor is unchanged
tensor

In [ ]:
# To change tensor
tensor = tensor + 10
tensor

In [ ]:
# Subtraction if you want
tensor - 10

In [ ]:
# We can use buit-in functions for muplitiplication
tf.math.multiply(tensor, 10)

In [ ]:
tensor

**Matrix Multiplication**

In machine learning, matrix multiplaction is one of the most common practice. In the above we have done element-wise operations.

There are two rules our tensors (or matrices) need to fulfil if we're going to matrix multiply them:

1. The inner dimensions must match.  
2. The reulting matrix has the shape of the inner dimension



In [ ]:
# Matrix multiplication
print(tensor)
tf.matmul(tensor, tensor)

In [ ]:
# Matrix multiplication with python operator "@"
tensor * tensor

In [ ]:
tensor @ tensor

In [ ]:
# Create tensor of (3,3) and (3,2)
A = tf.constant([[1, 2, 5],
                 [7, 2, 1],
                 [3, 3, 3]])
B = tf.constant([[3, 5],
                 [6, 7],
                 [1, 8]])
A, B

In [ ]:
tf.matmul(A, B)

In [ ]:
A @ B

**The dot product**

Matrix multiplication is also referred to as dot product

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`

📚**Resource:** https://matrixmultiplication.xyz/

In [ ]:
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])
X, Y

In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

In [ ]:
# Perform metrix multipplication between X and Y (transpose)
tf.tensordot(X, tf.transpose(Y), axes=1)

In [ ]:
# Perform matrix multiplication between X and Y (reshape)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

In [ ]:
# check the values of Y, reshape Y and transpose Y
print("Normal Y: ")
print(Y, "\n")

print("Y reshaped to (2, 3): ")
print(tf.reshape(Y, shape=(2, 3)), "\n")

print("Y transposed: ")
print(tf.transpose(Y))

In [ ]:
tf.matmul(X, tf.transpose(Y))

Generally, when performing matrix multiplication on two tensors and one of the axes donsn't line up, you will transpose (rather than reshape) one of the tensors to satisfy the matrix multiplication rules.

### Changing the datatype of the tensor

In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7 , 7.4])
B.dtype

In [ ]:
C = tf.constant([7, 10])
C.dtype

In [ ]:
# Changing from float32 to float16
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

In [ ]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E, E.dtype

In [ ]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16, E_float16.dtype

### Aggregating Tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values

In [ ]:
# Get the absolute values
D = tf.constant([-7, -10])
D

In [ ]:
# Get the absolute values
tf.abs(D)

Let us go through the following form of aggregation

* Get the minimum
* Get the maximum
* Get the mean of tensor
* Get the sum of a tensor

In [ ]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

In [ ]:
tf.size(E), E.shape, E.ndim

In [ ]:
# Find minimum
tf.reduce_min(E)

In [ ]:
# Find the maximum
tf.reduce_max(E)

In [ ]:
# Find the mean
tf.reduce_mean(E)

In [ ]:
# Find the sum
tf.reduce_sum(E)

🛠️ **Exercise:**  Find the variance and standard deviation of our E tensor using TensorFlow methods

In [ ]:
tf.variance(E) # Won't work

In [ ]:
# To find variance, we need to acces to tensorflow_probability
import tensorflow_probability as tfp
tfp.stats.variance(E)

In [ ]:
tf.math.reduce_std(tf.cast(E, dtype=tf.float32))

In [ ]:
tf.math.reduce_std(E)

In [ ]:
# Find variance of E
tf.math.reduce_variance(tf.cast(E, dtype=tf.float16))

### Finding the positional maximum and minimum of tensor

In [ ]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

In [ ]:
# Find the positional maximum
tf.argmax(F)

In [ ]:
# Index on our largest value position
F[tf.argmax(F)]

In [ ]:
# Find the max value of F
tf.reduce_max(F)

In [ ]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

In [ ]:
# Find the positional minimum
tf.argmin(F)

In [ ]:
# Index on our smallest value position
F[tf.argmin(F)]

In [ ]:
# Find the min value of F
tf.reduce_min(F)

In [ ]:
# Check for equality
F[tf.argmin(F)] == tf.reduce_min(F)

### Squeezing a tensor (removing all single dimensions)

In [ ]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

In [ ]:
G.shape

In [ ]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

`tf.squeeze` removes all dimensions of 1

### One-hot encoding tensors

In [ ]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue,, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth=4)

In [ ]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="I love deep learning", off_value="I also like to jog")

### Squaring, Log and Square Root

In [ ]:
# Create a new tensor
H = tf.range(1, 10)
H

In [ ]:
# Square it
tf.square(H)

In [ ]:
# Square Root
tf.sqrt(H) # Will result in error (Check docs)

In [ ]:
# Square Root
tf.sqrt(tf.cast(H, dtype=tf.float32))

In [ ]:
# Take log
tf.math.log(H) # Will give you error (check docs)

In [ ]:
tf.math.log(tf.cast(H, dtype=tf.float32))

### Tensors and Numpy

TensorFlow interacts beautifully with NumPy arrays

🔑 **Note:** One of the main differences between TensorFlow tensor and a NumPy array is that a TensorFlow tensor can be run on a GPU or TPU

In [ ]:
# Create tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

In [ ]:
# Convert tensor back to a NumPy array
np.array(J), type(np.array(J))

In [ ]:
# Convert tensor J to a Numpy array
J.numpy(), type(J.numpy())

In [ ]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check dtype of each
numpy_J.dtype, tensor_J.dtype

⚠️ To convert numpy to tensors, they may have a different data type and you might run into an error

### Finding access to GPUs

In [ ]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
!nvidia-smi

Mon Sep 20 16:54:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    69W / 149W |    123MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

> 🔑 **Note:** If you have access to CUDA-enabled GPU, TensorFlow will automatically use it whenever possible.